In [294]:
# goals
# variables
# constraints
# algos:
#   - GRG non linear (smooth nonlinear problems)
#   - LP Simplex (linear)
#   - Evolutionary (non-smooth problems)

# objective:
#    - even teams scores

# variables:
#    - Team size
#    - # of players
#    - player score (id, score)

# constraints:
# 1. player can be only on one team
# 2. team can be less but not more than max allowed
# 3. All players have to be used

#https://github.com/nicknochnack/LinearProgrammingBasics/blob/master/LP%20Notebook.ipynb


In [295]:
from sympy.interactive import printing
printing.init_printing(use_latex=True)
from sympy import Eq, solve_linear_system, Matrix
from numpy import linalg
import numpy as np
import sympy as sp
import csv
import pandas as pd
from pydantic import BaseModel
from enum import Enum, auto
from typing import List

In [296]:
data = pd.read_excel("test_data_Division.U05B.xlsx", index_col=0, dtype={"row#":int, "player_number":int, "player_name":str, "player_division":str, "player_score": int})
data_pd = pd.DataFrame(data[["player_number","player_division", "score_reduced"]])

print(len(data_pd))

56


### Variables

In [297]:
import random as rnd

def shuffle(roster: pd.DataFrame, **kwargs) -> pd.DataFrame:
    data = roster.sample(frac=1)
    return data

def slicer(data: pd.DataFrame, slices: int, size: int) -> pd.DataFrame:
    for i in range(0, (slices * size) - size, size):
        slice = pd.DataFrame(data.iloc[i:i+size])
        slice["slot#"]= int(i/size)
        yield slice

def generator(slices: int, size: int, **kwargs) -> List[pd.DataFrame]:
     teams = [i for i in slicer(shuffle(data_pd), slices, size)]
     return teams

In [372]:
league_teams = 8
players_per_team = 7
teams = generator(league_teams, players_per_team)
roster_mean = data_pd["score_reduced"].mean()

print("roster_mean", roster_mean)

means =[]
for t in teams:
    t_mean = t["score_reduced"].mean()
    means.append( { "team": t.iloc[0]["slot#"], "mean": t_mean, "diff": (roster_mean-t_mean)  } )

means_pd = pd.DataFrame(means)
stdev = means_pd["mean"].std() # <-- minimize this
print("stdev", stdev)

roster_mean 0.6103571428571429
stdev 0.09921362331608659


### Constraints

#### Goal